↓以下の数行でデータセットと訓練済みモデルをダウンロードし、ファインチューニングする

In [5]:
#id first_training
#caption Results from the first training
# CLICK ME
from fastai.vision.all import *
path = untar_data(URLs.PETS)/'images'

def is_cat(x): return x[0].isupper()
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224))

learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

/home/student/my_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/student/my_env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/student/my_env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /home/student/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 83.3M/83.3M [00:00<00:00, 107MB/s]


epoch,train_loss,valid_loss,error_rate,time
0,0.166709,0.026716,0.009472,02:07


epoch,train_loss,valid_loss,error_rate,time
0,0.054124,0.011653,0.003383,01:04


jupyterlab はipywidgetsを入れないといけない。GUIなのでまたあとで入れておく

In [6]:
uploader=widgets.FileUpload()
uploader

NameError: name 'widgets' is not defined

In [9]:
uploader = SimpleNamespace(data = ['fastbook-master/images/chapter1_cat_example.jpg'])

In [10]:
img = PILImage.create(uploader.data[0])
is_cat,_,probs = learn.predict(img)
print(f"Is this a cat?: {is_cat}.")
print(f"Probability it's a cat: {probs[1].item():.6f}")

Is this a cat?: True.
Probability it's a cat: 1.000000


最初のコード解説  

In [ ]:
from fastai.vision.all import *
#vision ライブラリをすべてインポート。これはCVモデルに必要な関数やクラス
path = untar_data(URLs.PETS)/'images'
#データダウンロード、解凍をして解凍場所をPathオブジェクトとして返す

def is_cat(x): return x[0].isupper()
#猫かどうかを示すラベルを返す関数（これはデータセット依存）。次の行で使う。このデータは文頭大文字なら猫
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224))
#データセットについての指示。データのロードを行う。同時に、ラベルの取得方法も指示。出来合いのラベル生成関数も多くある
#最後のはTransformで、item_tfmsは項目ごと変換だが基本はbatch_tfmsの方を多用する。出来合いのtransformも多くあるし自作でもいい
#from_name_funcは、「ファイル名に」関数（ここではis_cat）を適用するとラベルを得られることを示す
#valid_pctは検証セットの割合。選抜のシードを固定すること。

learn = vision_learner(dls, resnet34, metrics=error_rate)
#モデルを指定している。cnn_learnerとかもある。resnetはアーキテクチャの種類。第一引数がデータ。あとメトリック。error_rate以外にもいろいろ用意されてる
#pretraind引数もある。デフォルトでTrue
learn.fine_tune(1)
#fitではなく、もとのモデルをなるべく変えないやり方。引数がいろいろある

If you ever have any questions about a fastai method, you should use the function `doc`, passing it the method name:

```python
doc(learn.predict)
```

This will make a small window pop up with content like this:

<img src="images/doc_ex.png" width="600">

In [14]:
doc(ImageDataLoaders)

テーブルデータのdataloader部分。

In [ ]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
    cat_names = ['workclass', 'education', 'marital-status', 'occupation',
                 'relationship', 'race'],
    cont_names = ['age', 'fnlwgt', 'education-num'],
    procs = [Categorify, FillMissing, Normalize])

In [ ]:
learn.fit_one_cycle(3)
#事前学習済みモデルがないときによく使われる。でもfine_tuneのがうまく行くときもある

chap2

ある関数がどこで定義されているかわからなかったら、セルに素で置くことでわかる  
??をつけるとヘルプが見れる。　docと違ってfastai限定ではない

In [20]:
ImageDataLoaders

fastai.vision.data.ImageDataLoaders

In [21]:
??verify_images

Signature: verify_images(fns)
Source:   
def verify_images(fns):
    "Find images in `fns` that can't be opened"
    return L(fns[i] for i,o in enumerate(parallel(verify_image, fns)) if not o)
File:      ~/my_env/lib/python3.8/site-packages/fastai/vision/utils.py
Type:      function


mapメソッドは引数の関数がコレクションの各要素に適用される

次の%debug でデバッグ。便利

DataloadersはDataloaderをtrainとvalidに分け、モデルに食わせるための重要なクラス  
DataBlockは細かくカスタマイズできる  
  
他にもいろいろ細かいことはあるが、fastaiに限定しすぎだし飛ばしておく